# Training Suburban Shops

In [1]:
import pandas as pd
import numpy as np


pd.set_option('display.max_columns', None)

from datetime import datetime, timedelta, time

import pyodbc
import urllib
import sqlalchemy as sa

quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};"
                      "Server=ssqlpaazu01;"
                      "Database=Pret_Predictive;"
                      "Trusted_Connection=yes;"
                      )
engine = sa.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))


# Imports
from xgboost import XGBRegressor

import pickle

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys
sys.path.append('C:\\Users\\nirun\\Desktop\\Data Science\MTS Models\\SAV Build\\Model_Training_Functions')
from Model_Training_Functions import * 

In [3]:
# Load in the Table
food_q_script = '''

SELECT
    a.*,
    B.actual_retail_sales_net,
    d.cluster,
    d.description,
    E.LAUNCH_NAME AS quarter_label

FROM DATA.stage_till_data_MTS_agg as a
    LEFT JOIN DATA.stage_sales_history_day_aggregate AS B ON A.shop_id = B.shop_id AND a.t_date = B.date
    LEFT JOIN [DATA].[PY_SALES_FORECAST_CLUSTERS] as d on d.shop_id = A.shop_id
    LEFT JOIN (
                SELECT DISTINCT
                    A.DATE,
                    B.LAUNCH_NAME
                FROM data.stage_calendar AS A
                    INNER JOIN (
                                select
                                    launch_name,
                                    START_DATE,
                                    END_DATE
                                from warehouse.range_tool.dbo.[vw_launch_date] as a
                            ) AS B ON CAST(A.date AS DATE) BETWEEN CAST(B.START_DATE AS DATE) AND CAST(B.END_DATE AS date)
                ) AS E ON E.date = a.t_date
WHERE
    t_date BETWEEN '20171206' AND '20190610'
and
    cluster = 3
    
ORDER BY t_date

'''

suburban_initial = pd.read_sql(food_q_script, engine)
# Change column to datetime
suburban_initial.t_date = pd.to_datetime(suburban_initial.t_date)
print(suburban_initial.shape)
suburban_initial.head()

(748499, 13)


,shop_id,t_date,weekend_flag,day_of_week_no,day_of_week,bank_holiday_flag,week_day_description,product_id,qty,actual_retail_sales_net,cluster,description,quarter_label
0,10436,2017-12-06,0,4,Wednesday,0,Weekday,6479,10,4625.53,3,Shopping Centre,Christmas 2017
1,54,2017-12-06,0,4,Wednesday,0,Weekday,5676,45,17765.47,3,Shopping Centre,Christmas 2017
2,10550,2017-12-06,0,4,Wednesday,0,Weekday,5130,5,3658.74,3,Shopping Centre,Christmas 2017
3,10242,2017-12-06,0,4,Wednesday,0,Weekday,5773,9,3462.59,3,Shopping Centre,Christmas 2017
4,91,2017-12-06,0,4,Wednesday,0,Weekday,6913,2,6361.89,3,Shopping Centre,Christmas 2017


In [4]:
# Sales table
sales_script = '''select shop_id, date as t_date,shop_name, school_holiday  from [dbo].[vw_PY_sales_forecast_model] 
where date >= '2017-12-06' 


order by date, shop_name'''

# Load the table
sales_history = pd.read_sql(sales_script, engine)
# rename column

print(sales_history.shape)
sales_history.head()

(215166, 4)


,shop_id,t_date,shop_name,school_holiday
0,10285,2017-12-06,"Aberdeen, Bon Accord",0.0
1,10324,2017-12-06,"Aberdeen, Union Street",0.0
2,108,2017-12-06,Albemarle Street,0.0
3,10191,2017-12-06,Aldermanbury Square,0.0
4,10215,2017-12-06,Aldersgate,0.0


### Now I can use functions to clean table

In [5]:
%%time
# Now run the outlier detector
suburban_initial = range_finder(suburban_initial)

# Now detect outliers
range_suburban = find_outliers(suburban_initial)

The outlier ratio for the table is 6.143762383116076 percent
Wall time: 40.2 s


### Create the base table

In [6]:
# First Get rid out find_outliers
base_table = range_suburban.loc[range_suburban.outlier == 0]

# Add in new qty columns
base_table['new_qty'] = base_table.qty.map(lambda x: 4 if x <= 3 else x)

# Take a look
base_table.head(2)

,shop_id,t_date,weekend_flag,day_of_week_no,day_of_week,bank_holiday_flag,week_day_description,product_id,qty,actual_retail_sales_net,cluster,description,quarter_label,no_days_sold,shop_trading_days,sales_vol_percent,out_of_range,in_range_mean,out_of_range_mean,days_sold_range,days_sold_out_of_range,outlier,new_qty
0,10436,2017-12-06,0,4,Wednesday,0,Weekday,6479,10,4625.53,3,Shopping Centre,Christmas 2017,17,19,89.5,0,11.036145,NaN,55.963855,NaN,0,10
1,10436,2017-12-07,0,5,Thursday,0,Weekday,6479,14,5325.19,3,Shopping Centre,Christmas 2017,17,19,89.5,0,11.036145,NaN,55.963855,NaN,0,14


In [7]:
%%time
# Make a feature table
feature_suburban = mts_feature_builder(base_table, target='new_qty')
print(feature_suburban.shape, base_table.shape)

OK to process
(702513, 35) (702513, 23)
Wall time: 1min 5s


In [8]:
# Now add in weather
feature_suburban = weather_map(feature_suburban)
print(feature_suburban.shape)
feature_suburban.head()

(702513, 46)


,shop_id,t_date,weekend_flag,day_of_week_no,day_of_week,bank_holiday_flag,week_day_description,product_id,qty,actual_retail_sales_net,cluster,description,quarter_label,no_days_sold,shop_trading_days,sales_vol_percent,out_of_range,in_range_mean,out_of_range_mean,days_sold_range,days_sold_out_of_range,outlier,new_qty,rolling_week_sales,week_old,weekly_trend,same_last_4_days,same_last_6_days,rolling_4_days,rolling_6_days,rolling_4_max,rolling_6_max,rolling_4_min,rolling_6_min,daily_trend,SiteUID,ForecastDate,MaxTemperature,MinTemperature,FeelsLikeTemperature,SunshineHours,WeatherDescriptor,ForecastSiteUID,Weather_Code,Temp Range,temp_code
0,10436,2017-12-06,0,4,Wednesday,0,Weekday,6479,10,4625.53,3,Shopping Centre,Christmas 2017,17,19,89.5,0,11.036145,NaN,55.963855,NaN,0,10,10.0,NaN,1.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,350611,2017-12-06,10.0,8.0,6.7,0.0,Overcast,350611.0,2.0,5 to 10,5.0
1,10436,2017-12-07,0,5,Thursday,0,Weekday,6479,14,5325.19,3,Shopping Centre,Christmas 2017,17,19,89.5,0,11.036145,NaN,55.963855,NaN,0,14,14.0,NaN,1.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,1.0,350611,2017-12-07,12.0,5.0,5.7,3.0,Sunny spells,350611.0,3.0,5 to 10,5.0
2,10436,2017-12-08,0,6,Friday,0,Weekday,6479,11,5279.70,3,Shopping Centre,Christmas 2017,17,19,89.5,0,11.036145,NaN,55.963855,NaN,0,11,11.0,NaN,1.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,1.0,350611,2017-12-08,5.0,1.0,-2.0,1.5,Cloudy,350611.0,0.0,-5 to 0,-5.0
3,10436,2017-12-11,0,2,Monday,0,Weekday,6479,6,5073.04,3,Shopping Centre,Christmas 2017,17,19,89.5,0,11.036145,NaN,55.963855,NaN,0,6,6.0,NaN,1.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,1.0,350611,2017-12-11,1.0,-4.0,-5.2,3.0,Light snow,350611.0,8.0,-10 to -5,-10.0
4,10436,2017-12-12,0,3,Tuesday,0,Weekday,6479,9,5208.24,3,Shopping Centre,Christmas 2017,17,19,89.5,0,11.036145,NaN,55.963855,NaN,0,9,10.0,NaN,1.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,350611,2017-12-12,5.0,-2.0,-1.8,1.5,Cloudy,350611.0,0.0,-5 to 0,-5.0


In [9]:
# Merge in
print(feature_suburban.shape)
feature_suburban = feature_suburban.merge(sales_history, on = ['shop_id', 't_date'], how = 'left')
print(feature_suburban.shape)

(702513, 46)
(702513, 48)


## Now train and test the model

In [10]:
#FEature columns
feature_cols = ['shop_id', 'shop_name','product_id', 't_date', 'weekend_flag', 'qty', 'rolling_week_sales', 'weekly_trend',
                'same_last_4_days', 'same_last_6_days', 'daily_trend', 'out_of_range', 'in_range_mean', 'outlier',
                'product_name', 'category_desc', 'rolling_4_days', 'rolling_6_days', 'day_of_week_no', 'new_qty',
                'rolling_4_max', 'rolling_6_max', 'rolling_4_min', 'rolling_6_min', 'bank_holiday_flag', 'school_holiday',
                'Weather_Code', 'temp_code']

In [11]:
#Removal cols
removal_cols = ['qty', 't_date', 'in_range_mean', 'outlier', 'product_name', 'category_desc', 'new_qty', 'shop_name']

In [12]:
%%time
# Set up to test the function
Tester_final = Model_Tester(feature_cols, cols_to_remove=removal_cols, target_col='new_qty')

# Produce Dataframe
final_test = Tester_final.train_and_test(feature_suburban, fill_zero = False, clean_table = False, round_val = True)

# Get MAPE
print(Tester_final.MAPE_score)

# Get accuracy
print(Tester_final.Accuracy_score)

final_test.head()

0.2628045354331632
0.6452022196769225
Wall time: 46.9 s


In [13]:
Tester_final.xg_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [14]:
# Save the model
pickle.dump(Tester_final.xg_model, open('Shopping_Centre_feature_model.sav', 'wb'))

## Test

In [15]:
## Now try the model compared to teh one used over the weekend

# Load the feature table
weekend_features = pd.read_sql('select * from [DATA].[stage_PY_MTS_feature_table_iteration_2]', engine)

# Do datetime
weekend_features.t_date = pd.to_datetime(weekend_features.t_date)

# Add extra column of zeros
weekend_features['out_of_range'] = 0

print(weekend_features.shape)
weekend_features.tail()

print(weekend_features.t_date.min())

# Merge in
print(weekend_features.shape)
weekend_features = weekend_features.merge(sales_history, on = ['shop_id', 't_date'], how = 'left')
print(weekend_features.shape)


(2074810, 40)
2019-03-29 00:00:00
(2074810, 40)
(2074810, 42)


In [16]:

### Now run some tests

# Make modelling columns - 
modelling_cols = [x for x in feature_cols if x not in removal_cols]

# Load models
weekend_model = pickle.load(open('Shopping_Centre_feature_model.sav', 'rb'))
suburban_model = pickle.load(open('worker_feature_model.sav', 'rb'))

# Now get predictions
weekend_pred = weekend_model.predict(weekend_features[modelling_cols])
weekend_features['weekend_predictions'] = weekend_pred

# Now get predictions
suburban_pred = suburban_model.predict(weekend_features[modelling_cols])
weekend_features['suburban_predictions'] = suburban_pred

weekend_features.tail()

# Round the values
weekend_features.weekend_predictions = weekend_features.weekend_predictions.map(lambda x: round(x, 0))
weekend_features.suburban_predictions = weekend_features.suburban_predictions.map(lambda x: round(x, 0))

weekend_features['weekend_MAPE'] = abs(weekend_features.new_qty - weekend_features.weekend_predictions) / weekend_features.new_qty
weekend_features['suburban_MAPE'] = abs(weekend_features.new_qty - weekend_features.suburban_predictions) / weekend_features.new_qty

# Get rid of infintiy
weekend_features = weekend_features.applymap(lambda x: np.nan if x == np.inf else x)

# Look at MAPE
print(weekend_features.weekend_MAPE.mean(), weekend_features.suburban_MAPE.mean())

# Now accuracy
weekend_features['weekend_acc'] = weekend_features.apply(lambda x: new_accuracy_score_new_qty(x), axis=1)
weekend_features['suburban_acc'] = weekend_features.apply(lambda x: new_accuracy_score_new_qty(x, target_pred='suburban_predictions'), axis=1)

# Now look at accuracy
print(weekend_features.weekend_acc.mean(), weekend_features.suburban_acc.mean())

# Suburban cluster shows improvement!
weekend_features.loc[weekend_features.qty > 2].groupby('description')[['weekend_acc', 'suburban_acc']].mean()

help(Model_Tester)

0.23850459133269045 0.21844589667537598


KeyboardInterrupt: 